In [1]:
import numpy as np
from scipy import sparse
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Concatenate

In [2]:
print(keras.__version__)

2.1.6-tf


## - Full model

In [3]:
def make_model():

    position_1=Input((774,))
    position_2=Input((774,))
    
    model_encode=Sequential()
    
    model_encode.add(Dense(units=700, input_dim=774, use_bias=False,
                           kernel_initializer=keras.initializers.he_normal(seed=None)))
    model_encode.add(BatchNormalization())
    model_encode.add(Activation("relu"))
    
    model_encode.add(Dense(units=500, use_bias=False,
                           kernel_initializer=keras.initializers.he_normal(seed=None)))
    model_encode.add(BatchNormalization())
    model_encode.add(Activation("relu"))
    
    model_encode.add(Dense(units=400, use_bias=False,
                           kernel_initializer=keras.initializers.he_normal(seed=None)))
    model_encode.add(BatchNormalization())
    model_encode.add(Activation("relu"))           
              
    model_encode.add(Dense(units=300, use_bias=False,
                           kernel_initializer=keras.initializers.he_normal(seed=None)))
    model_encode.add(BatchNormalization())
    model_encode.add(Activation("relu"))
    
    encoded_position_1=model_encode(position_1)
    encoded_position_2=model_encode(position_2)
    
    prediction=Concatenate()([encoded_position_1, encoded_position_2])
    
    prediction=Dense(units=800, use_bias=False,
                     kernel_initializer=keras.initializers.he_normal(seed=None))(prediction)
    prediction=BatchNormalization()(prediction)
    prediction=Activation("relu")(prediction)
    
    prediction=Dense(units=600, use_bias=False,
                     kernel_initializer=keras.initializers.he_normal(seed=None))(prediction)
    prediction=BatchNormalization()(prediction)
    prediction=Activation("relu")(prediction) 
    
    prediction=Dense(units=400, use_bias=False,
                     kernel_initializer=keras.initializers.he_normal(seed=None))(prediction)
    prediction=BatchNormalization()(prediction)
    prediction=Activation("relu")(prediction)
    
    prediction=Dense(units=100, use_bias=False,
                     kernel_initializer=keras.initializers.he_normal(seed=None))(prediction)
    prediction=BatchNormalization()(prediction)
    prediction=Activation("relu")(prediction)
    
    prediction=Dense(units=1, use_bias=False)(prediction)
    prediction=BatchNormalization()(prediction)
    prediction=Activation("sigmoid")(prediction)
    
    model=Model(inputs=[position_1, position_2], outputs=prediction)
    
    return model

In [4]:
model = make_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 774)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 774)          0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 300)          1219400     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 600)          0           sequential[1][0]                 
          

In [5]:
adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [6]:
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

## - Load test data

In [7]:
data_test = sparse.load_npz("data_test.npz")
X_test_1 = data_test[0:10**5,0:774]
X_test_2 = data_test[0:10**5,774:1548]
y_test = data_test[0:10**5,1548]
del data_test

## - Train

In [8]:
for i in range(1,5):
    data = sparse.load_npz("data_" + str(i) + ".npz")
    X_train_1 = data[:,0:774]
    X_train_2 = data[:,774:1548]
    y_train = data[:,1548]
    del data
    model.fit([X_train_1, X_train_2], y_train, epochs=1, batch_size=512, validation_data=([X_test_1, X_test_2], y_test), verbose=1)
    model.save("model_" + str(i) + ".h5")

Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1889s 378us/step - loss: 0.4358 - acc: 0.7846 - val_loss: 0.4058 - val_acc: 0.8062
Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1878s 376us/step - loss: 0.3693 - acc: 0.8271 - val_loss: 0.3841 - val_acc: 0.8198
Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1875s 375us/step - loss: 0.3484 - acc: 0.8390 - val_loss: 0.3690 - val_acc: 0.8287
Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1882s 376us/step - loss: 0.3356 - acc: 0.8458 - val_loss: 0.3569 - val_acc: 0.8344


In [9]:
for i in range(5,7):
    data = sparse.load_npz("data_" + str(i) + ".npz")
    X_train_1 = data[:,0:774]
    X_train_2 = data[:,774:1548]
    y_train = data[:,1548]
    del data
    model.fit([X_train_1, X_train_2], y_train, epochs=1, batch_size=512, validation_data=([X_test_1, X_test_2], y_test), verbose=1)
    model.save("model_" + str(i) + ".h5")

Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1876s 375us/step - loss: 0.3401 - acc: 0.8436 - val_loss: 0.3404 - val_acc: 0.8439
Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1885s 377us/step - loss: 0.3214 - acc: 0.8538 - val_loss: 0.3314 - val_acc: 0.8482


In [10]:
for i in range(7,9):
    data = sparse.load_npz("data_" + str(i) + ".npz")
    X_train_1 = data[:,0:774]
    X_train_2 = data[:,774:1548]
    y_train = data[:,1548]
    del data
    model.fit([X_train_1, X_train_2], y_train, epochs=1, batch_size=512, validation_data=([X_test_1, X_test_2], y_test), verbose=1)
    model.save("model_" + str(i) + ".h5")

Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1886s 377us/step - loss: 0.3192 - acc: 0.8549 - val_loss: 0.3340 - val_acc: 0.8473
Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1885s 377us/step - loss: 0.3307 - acc: 0.8483 - val_loss: 0.3335 - val_acc: 0.8469


In [12]:
from tensorflow.keras import backend as K

In [13]:
K.get_value(model.optimizer.lr)

0.001

In [14]:
K.set_value(model.optimizer.lr,0.0001)

In [15]:
K.get_value(model.optimizer.lr)

1e-04

In [16]:
for i in range(9,21):
    data = sparse.load_npz("data_" + str(i) + ".npz")
    X_train_1 = data[:,0:774]
    X_train_2 = data[:,774:1548]
    y_train = data[:,1548]
    del data
    model.fit([X_train_1, X_train_2], y_train, epochs=1, batch_size=512, validation_data=([X_test_1, X_test_2], y_test), verbose=1)
    model.save("model_" + str(i) + ".h5")

Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1880s 376us/step - loss: 0.3091 - acc: 0.8598 - val_loss: 0.3223 - val_acc: 0.8526
Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1879s 376us/step - loss: 0.3049 - acc: 0.8619 - val_loss: 0.3195 - val_acc: 0.8539
Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1880s 376us/step - loss: 0.3217 - acc: 0.8528 - val_loss: 0.3211 - val_acc: 0.8540
Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1879s 376us/step - loss: 0.3124 - acc: 0.8576 - val_loss: 0.3203 - val_acc: 0.8543
Train on 5000000 samples, validate on 100000 samples
Epoch 1/1
5000000/5000000 [==============================] - 1886s 377us/step - loss: 0.3074 - acc: 0.8603 - val_loss: 0.3200 - val_acc: 0.8549
Train on 500000